In [4]:
import sys
sys.path.append(r'D:\学习\微热点大赛\DataMiningForCompetation')
from db import MyDB
#加载数据库
db = MyDB(host='182.61.54.181', user='root', password='yW88224646!', db='WRD')

In [36]:
#加载使用库
import pandas
import os
import numpy
import jieba.posseg as pseg
import re
from tqdm import tqdm
import json

In [37]:
# [是否为默认文本,内容文本,内容表情,地域,城市,性别,粉丝数,微博数,转,评,赞,话题,微博情绪,精准地域,中图地址,
# MD5-作者ID,MD5-mid,MD5-根微博mid,MD5-根微博用户UID,MD5-父微博ID,MD5-父微博用户ID]
targetRoot = r'E:\Download\数据大赛\选题1\初步分词\\'
sourceRoot = r'E:\Download\数据大赛\选题1\微博'
needPartOfSpeech = ['a','ad','an']
tl1 = []
emotion = re.compile(r'(\[.*?\])')
tag = re.compile(r'(#.*?#)')
tags = []
emotions = []
files = None
for root,dirs,fs in os.walk(sourceRoot):
    files = fs
    break
for file in files:
    fileAbsolutePath = sourceRoot+'\\'+file
    dataFile = pandas.read_csv(fileAbsolutePath,encoding='gb18030')
    tempList = []
    for item in tqdm(dataFile.values):
        temp = [int(item[0] == '转发微博 '),json.dumps(emotion.findall(item[0])), json.dumps(tag.findall(item[0])), item[0]]
        for e in temp[1]:
            temp[3] = temp[3].replace(e,'')
        for t in temp[2]:
            temp[3] = temp[3].replace(t,'')
        if temp[3] == '转发微博 ':
            temp[3] = ''
        tempWords = []
        words = pseg.cut(temp[3],use_paddle=True)
        for word, flag in words:
            if flag in needPartOfSpeech:
                tempWords.append(word)
        temp[3] = json.dumps(tempWords)
        temp.append(int(item[1] == '敏感'))
        temp.append(int(item[2] == '原创'))
        for i in range(3,25):
            if i == 9 or i == 4:
                continue
            temp.append(item[i])
        tempList.append(temp)
    tempList = pandas.DataFrame(tempList)
    tempList.columns = ['是否为默认文本','内容表情','内容标签','过滤后内容分词json串','是否敏感','是否原创','发布日期','认证类型','地域','城市','性别','粉丝数','微博数','转','评','赞','话题','微博情绪','精准地域','中图地址',
      'MD5-作者ID','MD5-mid','MD5-根微博mid','MD5-根微博用户UID','MD5-父微博ID','MD5-父微博用户ID']
    tempList.to_csv(targetRoot+file,encoding='gb18030')

  0%|                                                                                       | 0/221902 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\17797\AppData\Local\Temp\jieba.cache
Loading model cost 0.709 seconds.
Prefix dict has been built successfully.
100%|█████████████████████████████████████████████████████████████████████████| 136483/136483 [04:01<00:00, 566.29it/s]
